In [704]:
# # -*- coding: utf-8 -*-
# from ckiptagger import data_utils # first use ckip, need to download data file
# data_utils.download_data_gdown("./")
from ckiptagger import WS, POS, NER, construct_dictionary

ws = WS("./ckip_data") # word segment
pos = POS("./ckip_data") 
ner = NER("./ckip_data") # entity recognition

# # extend dictionary
# word_to_weight = {
#     "緯來體育台": 1,
#     "鬼金棒": 2,
#     "海底撈": 2
# }
# dictionary = construct_dictionary(word_to_weight)
# print(dictionary)



import pandas as pd
import numpy as np
import time
import math
import re
import demoji
# demoji.download_codes()
from selenium.webdriver import ActionChains # activate esc key
from selenium.webdriver.common.keys import Keys # activate esc key

#====== import jieba dict =====
import jieba
jieba.set_dictionary('dataset/dic_trad/dict.txt.big')
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt')


#====== selenium cathc NoSuchElementException ======
from selenium.common.exceptions import NoSuchElementException   
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

Building prefix dict from /home/alvin/essay/dataset/dic_trad/dict.txt.big ...
Loading model from cache /tmp/jieba.ucbd952ca8a83da26cb61ea55c46a2e3b.cache
Loading model cost 0.711 seconds.
Prefix dict has been built successfully.


In [858]:
#====== get antusd senti dict ======
colNames = ['Word','Score','Pos','Neu','Neg','Non','Not']
antusd = pd.read_csv('dataset/ANTUSD_traditional/opinion_word.csv', names = colNames) 
word_senti_score = dict(zip(antusd['Word'],antusd['Score'])) # find score via word, 27221 words

# print(antusd)
# print(word_senti_score)

In [315]:
def get_selenium_param(filename):
    paramList = []
    with open("dataset/ig_article/"+filename, "r") as f:
        for line in f:
            single_store_param = line.split(',')
            single_store_param[2] = single_store_param[2].replace('\n','')
            paramList.append(single_store_param)
#             senti_word = line.split()[0]
#             senti_score = float(line.split()[1])
#             word_senti_score[senti_word] = senti_score
#             print()
    return paramList

print(get_selenium_param('selenium_url'))


[['Ippudo TW 一風堂(台灣)', 'href="/explore/locations/13656102/ippudo-tw/"', 'href="/p/B-WxOwxpv18/"']]


In [326]:
get_selenium_param('selenium_url')

[['花月嵐拉麵 松山車站店',
  'href="/explore/locations/227701777/"',
  'href="/p/B-hSxkCJNrs/"'],
 ['#花月嵐拉麵', 'href="/explore/tags/花月嵐拉麵/"', 'href="/p/B-HaG68n0fV/"']]

In [718]:
#====== extend senti. dict ======
def slef_extend_senti_dict(word_senti_score):
    with open("dataset/dic_trad/extend_senti_score_dict.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score


#====== extend senti. dict ======
def slef_extend_senti_dict_hotpot(word_senti_score):
    with open("dataset/dic_trad/extend_senti_score_dict_hotpot.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score    
        


word_senti_score['good'] = 0.0550611
word_senti_score['棒'] = 0.1
word_senti_score['中規中矩'] = -0.05
word_senti_score['感覺'] = 0.02
word_senti_score['覺得'] = 0.02
word_senti_score['第一'] = 0.2
word_senti_score['虛'] = -0.2
word_senti_score['爛'] = -0.2
word_senti_score['差'] = -0.2
word_senti_score['好差'] = -0.2
word_senti_score['久'] = -0.1
word_senti_score['太久'] = -0.4
word_senti_score['厲害'] = 0.2
word_senti_score['熱死'] = -0.2
word_senti_score['排隊'] = -0.1
word_senti_score['不高'] = -0.3
word_senti_score['鹹'] = -0.1
word_senti_score['好鹹'] = -0.2
word_senti_score['太鹹'] = -0.3
word_senti_score['沒'] = -0.1
word_senti_score['普通'] = -0.02
word_senti_score['不'] = -0.05
slef_extend_senti_dict(word_senti_score)
slef_extend_senti_dict_hotpot(word_senti_score)

In [464]:
#====== test ckip word segment ======

# text = '傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。'
# text = '"鬼金棒，我就跟！就這樣，我又跟我的前衣食父母吃飯！連吃兩天拉麵🍜一個胖的節奏#鬼金棒拉麵#辣正常麻增量#結果真的麻到我頭皮發麻 我到底要不要改機票呢😏"'
# text = '期盼那麼久才吃到，果然不負期望，無論是湯頭還是肉片都是最佳狀態，太享受了'
text = '好久不見！，體力一年不如一年，#20191101，服務態度超優的，海鮮😋,海底撈圈字遊戲送一盤肉,👍👍👍撈撈撈，💋💋💋,排隊排好久 超值得, 我要再來刷二訪 😋， 以愛吃重口味的我來說 麵條的硬度很讚 乾, 想吃很膩害的溏心蛋 這家沾麵真的很優,湯頭也很濃 我不愛沾麵 難波萬台灣美食成為我的愛店之一👍🏻好多人，不知道要排多久,麻的味道超級棒,推推推,湯頭還是很棒，豆芽菜的鑊氣沒了很難過，大概一陣子不會回訪了,good excelent,熱死第一名,可惜還是沒把湯喝完,應該很普通,不是我的最愛。'


ws_results = ws([text])
# pos_results = pos(ws_results)
# ner_results = ner(ws_results, pos_results)

print(ws_results)
# print(pos_results)
# for name in ner_results[0]:# print entity
#     print(name)

[['好', '久', '不', '見', '！', '，', '體力', '一', '年', '不如', '一', '年', '，', '#20191101', '，', '服務', '態度', '超優', '的', '，', '海鮮', '😋', ',', '海', '底', '撈圈字', '遊戲', '送', '一', '盤', '肉', ',', '👍👍👍', '撈撈撈', '，', '💋💋💋', ',', '排隊', '排好', '久', ' ', '超', '值得', ',', ' ', '我', '要', '再', '來', '刷', '二', '訪', ' 😋', '，', ' ', '以', '愛', '吃重', '口味', '的', '我', '來', '說', ' ', '麵條', '的', '硬度', '很', '讚', ' ', '乾', ',', ' ', '想', '吃', '很', '膩害', '的', '溏心蛋', ' ', '這', '家', '沾麵', '真的', '很', '優', ',', '湯頭', '也', '很', '濃', ' ', '我', '不', '愛', '沾麵', ' ', '難波萬', '台灣', '美食', '成為', '我', '的', '愛店', '之', '一👍🏻', '好多', '人', '，', '不', '知道', '要', '排', '多久', ',', '麻', '的', '味道', '超級棒', ',', '推推推', ',', '湯頭', '還是', '很', '棒', '，', '豆芽菜', '的', '鑊氣', '沒', '了', '很', '難過', '，', '大概', '一陣子', '不會', '回訪', '了', ',', 'good excelent', ',', '熱死', '第一', '名', ',', '可惜', '還是', '沒', '把', '湯', '喝完', ',', '應該', '很', '普通', ',', '不', '是', '我', '的', '最愛', '。']]


In [674]:
#====== emoji_sentiment_dict ======
emoji_sentiment = pd.read_csv('dataset/dic_trad/Emoji_Sentiment.csv') # read Emoji_Sentiment.csv
emoji_sentiment.drop(columns=['Unnamed: 0'], inplace = True)
emoji_sentiment_dict = dict(zip(emoji_sentiment['Emoji'],emoji_sentiment['Sentiment score'])) # key,value

In [6]:
# word_pos = dict(zip(ws_results[0],pos_results[0])) # combine word and pos tag

In [383]:
#====== data preprocess ======
def data_preprocess(articles): 
    # only get string comment & preprocess fans, likes
    all_comments, assigned_index, fans_count, likes  = [],[],[],[]  # assigned_index: denote the original dataframe index of comments
    for i in range(len(articles['text'])):     # remove '\n'
        if type(articles['text'][i]) == str:   # remove non-string
            if articles['text'][i].find('\n'): # if contain '\n', replace with ','
                all_comments.append(articles['text'][i].replace('\n','，'))
            else:
                all_comments.append(articles['text'][i])
            assigned_index.append(i)
            
#             # get assigned fans_count
#             if type(articles['fans_count'][i]) == np.float64 and math.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif type(articles['fans_count'][i]) == np.float64 and np.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif str(articles['fans_count'][i]).find('萬') != -1:
#                 wan_index = articles['fans_count'][i].find('萬')
#                 transform_count = (float(articles['fans_count'][i][:wan_index])) * 10000
#                 fans_count.append(transform_count)
#             else:
#                 fans_count.append(int(articles['fans_count'][i]))
            fans_count = []

            # method2
            # get assigned likes
#             print(i)
            temp_like = articles['likes'][i]
            if type(temp_like) == np.float64 or type(temp_like) == np.int64:
                likes.append(temp_like)
            elif type(temp_like) == str:
                if temp_like.isdigit():
                    likes.append(float(temp_like))
                else:
                    likes.append(1)
            else:
                likes.append(temp_like)
            
            
            
            # method1
#             if np.isnan(articles['likes'][i]):
#                 likes.append(1)
#             else:
#                 likes.append(articles['likes'][i])
    
    return all_comments, assigned_index, fans_count, likes

In [790]:
#====== cal senti score by jieba ======
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict

def get_senti_score(text):
    '''calculate one sentence's score'''
    ws_results = [jieba.lcut(text)] # word segment
    # get sentiword in dict
    total_score = 0
    for word in ws_results[0]:
        if word in word_senti_score:
            score = word_senti_score[word]
            total_score += score 
            print(word,score) # print each senti word's senti score
    
    
    print('sent_score:',total_score)
    if total_score >= 1:
        return 1
    elif total_score <= -1:
        return -1
    else:
        return total_score

In [707]:
#====== cal senti score by ckip ======
def get_senti_score(text):
    ws_results = ws([text]) # word segment
    total_score = 0
    for word in ws_results[0]:
        if word in word_senti_score:
            score = word_senti_score[word]
            total_score += score 
            print(word,score) # print each senti word's senti score
    
    
    print('sent_score:',total_score)
    if total_score >= 1:
        return 1
    elif total_score <= -1:
        return -1
    else:
        return total_score
    
#====== get each comment's score ======
def get_each_comment_score(all_comments):
    all_comments_scores = []
    for single_comment in all_comments:
        print('\ncomment:',single_comment)
        all_comments_scores.append(get_senti_score(single_comment))
        
    return all_comments_scores


In [630]:
#====== cal store score ======      # not yet consider no comment post's effect (maybe effective) 
def get_store_score(weight, each_comment_score, assigned_index):
    '''store score = comment score * likes(weight) /all likes'''
    weighted_score = 0
    total_weight = 0
    for i in range(len(each_comment_score)):
        if not np.isnan(weight[i]):
            weighted_score += (weight[i] * each_comment_score[i])
            total_weight += weight[i] 
            
    store_score = weighted_score / total_weight

    return store_score


In [102]:
def output_csv(filename, data_list, write_method):

    import csv

    # 開啟輸出的 CSV 檔案
    with open(filename, write_method, newline='') as csvfile:
        writer  = csv.writer(csvfile)   # 建立 CSV 檔寫入器
        for row in data_list: # format: two dimension
            writer.writerow(row)
            

In [ ]:
#====== remove foreign language post ======
for i,content in zip(articles[articles['text'].notnull()]['text'].index,
                     articles[articles['text'].notnull()]['text']):
    if not re.search(r'[\u4e00-\u9fff]+', content) and not demoji.findall(content): # check contain chinese
        print(i,content)
        # drop assigned row
        articles.drop(i, inplace=True) 
        
# reset dataframe index
articles.reset_index(drop=True) 

In [687]:
text = '這是一場五感都被放大的饗宴。﻿，﻿，有硬度及Q勁的麵體，🐠很喜歡，搭配辣少麻少的濃厚湯頭，非常過癮！鬼金棒的辣和麻是一流的，入口先感受到辣，隨之而來的麻會讓你感受到舌尖的震顫，建議第一次吃、不是很敢吃麻辣的朋友跟🐠一樣選擇辣少麻少。﻿，﻿，鬼金棒的拉麵，就是這樣強勁有力。﻿，﻿，叉燒肉也和大多數的拉麵店不同，選用大塊的五花肉，乍看之下有點像控肉（但絕對不是），肥瘦相間、軟嫩好入口，有別於一般薄片叉燒，鬼金棒就是這樣的霸氣！﻿，﻿，最後額外提一下，店裡的冰開水不知道為什麼，特別甘甜好喝啊🤣﻿，﻿，﻿，-·-·-·-·-·-﻿，🔺用餐請遵守店內顧客須知，大家一起當優良好客人！﻿，🔺吃完麵再加白飯100分！但麵的份量本身就蠻大了，要自己斟酌哦！﻿，——————————，﻿，🌟如果喜歡濃厚拉麵，或者吃膩了一般口味的拉麵，來鬼金棒走一遭吧☺️﻿，﻿，——————————，🏠辣麻味噌拉麵 鬼金棒﻿，📍104台北市中山區長安西路19巷2弄35號（近捷運中山站）﻿，🕐平日11:30-14:00/17:00-20:30，假日11:30-20:30﻿，，#辣麻味噌拉麵 #鬼金棒 #鬼金棒カラシビ味噌ラーメン #拉麵 #叉燒肉 #捷運美食 #捷運中山站 #ramen #ramennoodles 當然是想到鬼金棒💕，麻的味道超級棒，讓你一口接一口🤤，肉的份量也很足👍，湯頭非常濃郁‼️，吃#nightmarket #taiwan #taipei #noodles #stinkytofu 🤤#squid #seafood #dumplings #ningxia #nightlife,豪豪ㄘ 嘴唇超麻～～～,敲好ㄘ🌶🌶🌶🌶🍜'

# print(ws([text]))
demoji.findall(text)
for emoji in demoji.findall(text):
    if emoji_sentiment_dict.get(emoji):
        print(emoji,emoji_sentiment_dict[emoji])

🐠 0.414
🌟 0.327
🏠 0.5
💕 0.632
📍 0.111
🍜 0.395
👍 0.521


In [685]:
emoji_sentiment_dict['🤤']

KeyError: '🤤'

In [939]:
jieba.lcut('喝起來更香濃愛吃重口味的我來說嗚嗚嗚嗚可惜吃到後面有點膩今天沾麵日！白湯&海湯沾麵🍜')

['喝',
 '起來',
 '更',
 '香濃',
 '愛吃',
 '重',
 '口味',
 '的',
 '我',
 '來說',
 '嗚嗚',
 '嗚嗚',
 '可惜',
 '吃',
 '到',
 '後面',
 '有點',
 '膩',
 '今天',
 '沾麵',
 '日',
 '！',
 '白湯',
 '&',
 '海湯',
 '沾麵',
 '🍜']

In [844]:
len(all_comments_scores)

50

In [771]:
articles[(articles['text']).isnull()]['text'] # get NaN text

3     NaN
8     NaN
15    NaN
19    NaN
26    NaN
27    NaN
37    NaN
38    NaN
Name: text, dtype: object

In [946]:
all_comments_scores

[1,
 0.7882361,
 0.4,
 0.2386419,
 0.15,
 0.9161983,
 0.3,
 1,
 0.8,
 0.697982833,
 1,
 1,
 1,
 1,
 0.529169,
 1,
 1,
 0.25,
 1,
 1,
 0.9564695000000001,
 0.67684395,
 0.2611116,
 0.85,
 0.25,
 0.65,
 1,
 0,
 1,
 1,
 1,
 1,
 0.4,
 1,
 1,
 1,
 0.75,
 0.013796867000000004,
 1,
 0.6000000000000001,
 0.20400079999999998,
 0.49684395,
 0.9057268,
 0.37684395,
 1,
 1,
 0,
 0.55,
 0.2,
 0.0,
 0.8,
 0.7325394000000001,
 1,
 1,
 0.4,
 -0.1,
 0.4,
 0.65,
 1,
 1,
 1,
 0.65,
 0.6731792000000001,
 1,
 0.35,
 1,
 0.44999999999999996,
 -0.6026039,
 1,
 0.5039873,
 1,
 0.6185088000000001,
 1,
 0,
 1,
 0.3,
 0,
 0.2,
 0.4,
 0.8,
 0.5796884,
 1,
 0.25,
 1,
 1,
 0.35,
 1,
 0.3,
 1,
 0.9905789000000002,
 1,
 0.6499999999999999,
 0,
 1,
 1,
 0.2,
 1,
 1,
 1,
 0,
 0.5833655829999999,
 0.8615198999999999,
 0.65,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0.1734641,
 0.8500000000000001,
 -0.10442180000000001,
 0.25,
 1,
 1,
 0.1,
 0.6283002329999998,
 1,
 1,
 1,
 1,
 0.21097580000000002,
 1,
 1,
 0.85]

In [964]:
#====== cal one store rating ======

#====== update senti_dict & jieba_dict ======
slef_extend_senti_dict(word_senti_score)
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict


# get articles, followers
# fileName = 'hotpot/article_海底撈火鍋'

# fileName = 'article_山嵐拉麵_公館店_台灣總店' # 85 post
# fileName = 'article_山嵐拉麵' # 33 post (remove)
# fileName = 'article_你回來啦 Okaeri 拉麵'# 96
# fileName = 'article_鬼金棒_中山北路' # 126
# fileName = 'article_麵屋壹の穴 ICHI' # 50 (remove)
fileName = 'article_麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店' # 106
# fileName = 'article_柑橘 Shinn' # 69


articles = pd.read_csv(open('dataset/ig_article/store_article/'+fileName+'.csv')) 

# remove '\n', delete non-string
all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

# get each comment's score
all_comments_scores = get_each_comment_score(all_comments)

# get store rating
# print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
store_rating = get_store_score(likes, all_comments_scores, assigned_index)
print('\nstore final score:',store_rating) # weight: likes
print('post count:',len(all_comments_scores))

# output ramen rating 
output_csv('ramen_rating.csv', [[fileName, store_rating]], 'a+')




comment: 秋刀魚拉麵🍜。
拉麵 0.15
🍜 0.15
sent_score: 0.3

comment: 目前在國內吃到最喜歡的沾麵😚😚😚
喜歡 0.875
沾麵 0.15
sent_score: 1.025

comment: 好久沒來了!!!!!!
sent_score: 0

comment: 完食，#ramen🍜 #menyaichi #tonkotsugyokai #karaage #完食 #穴穴你 #emptybowl
ramen 0.15
🍜 0.15
sent_score: 0.3

comment: 𝟚𝟘𝟙𝟡.𝟙𝟚.𝟘𝟙 #重溫前男友的口袋名單系列🍜，🍜麵屋壹の穴ICHI：煮干拉麵+雞叉燒，💰NT.250+50元，📍台北市大安區延吉街137巷22號，🚇鄰近捷運國父紀念館站1號出口，🥢梅漬聖女必推，⭐️3／5，-，#不負責任小心得🗣，壹の穴應該是前男友帶我去過所有拉麵店我最喜歡的了，分手後我自己也常常去吃，但平常都吃雞白湯；今天六點去排隊的時候老闆跟我說雞白湯賣完了⋯⋯最愛梅漬聖女小蕃茄今天也沒有⋯⋯整個很受打擊（心痛）。，但今天改口味嘗試吃了煮干拉麵，是沒有想像中的鹹，但還是更喜歡雞白湯啦。，-，#前男友小故事🐏，壹の穴是前男友帶我吃的第二間拉麵，第一間是鬼金棒。吃這兩間的時候我們還沒在一起，處於一種微妙的曖昧狀態。，要搭捷運的時候他問我今天想吃怎樣的拉麵，翻出他的口袋名單跟我一個一個介紹，最後選擇了壹の穴。，在捷運上他一口篤定的說，我一定會喜歡吃這間以及它的梅漬聖女，我半信半疑的說「真的嗎」，結果後來真的被他說中，我真的很喜歡這間的雞白湯還有梅漬聖女。，那時候他邊吃邊問我怎麼樣，我當時應該真的是眼冒愛心的說「好喜歡」。，印象中他笑得滿開心的，說著會幫我找到我最喜歡的拉麵店。我也就真的相信他會帶我去吃很多拉麵。，，有時候自己去吃壹の穴的時候還是不免會想起他，也會看著我們當時一起坐的位置發呆。說真的還是很想念他帶我來吃壹の穴的那一天。
重溫 0.0
🍜 0.15
🍜 0.15
拉麵 0.15
叉燒 0.0
出口 0.0
不負責任 -0.171182975
小心 0.0
應該 0.016946400000000004
所有 0.25781350000000003
拉麵 0.15
喜歡 0.875
分手 -0.133801
自己 0.0
常常

可以 0.1551797
發現 0.11151989999999998
完全 0.45943999999999996
沾麵 0.15
唯一 0.0540008
不同 0.0
流程 0.0
嘗試 0.05
知道 0.529169
直接 0.326906
其實 0.38113745
可以 0.1551797
特殊 0.2399749
不得不 0.0
控制 0.0
避免 0.0
完全 0.45943999999999996
蓋住 0.0
味道 0.0
算是 0.15
特殊 0.2399749
重點 0.1391674
根本 0.02277495
自助 0.4579737
加入 0.0625365
沾麵 0.15
當作 0.08317395
淨空 0.0
喜歡 0.875
覺得 0.5868026
好吃 0.37684395
相似 0.1846725
寧願 0.0
拉麵 0.15
可能 0.0
機會 0.3837311
嘗試 0.05
不能 -0.060934800000000004
差異 -0.57738575
覺得 0.5868026
值得 0.23
sent_score: 13.376151000000005

comment: 煮干好爽ㄛ
好爽 0.5
sent_score: 0.5

comment: #麵屋壹の穴ichi ，#乾擔擔麵 +豬叉燒，相見恨晚！！
叉燒 0.0
sent_score: 0.0

comment: 今天運氣蠻好的，完全沒排到隊就順利入座了，然後就開始排隊了XDDD，，目前是試營運，每天的麵條和湯頭都會做調整。今天吃的是咖哩和魚介豚骨兩種湯頭。
完全 0.45943999999999996
順利 0.0
開始 0.1162332
營運 0.0
調整 0.37553825
sent_score: 0.95121145

store final score: 0.6613204122823985
post count: 106


In [962]:
#====== MSE ======
# input store name: [a,b,c]
# output: [score.....]
def cal_all_store_rating(store_names):
    all_store_rating = []
    all_store_post_count = [] # store info dict.
    for i,name in enumerate(store_names):
        fileName = 'article_' + name
        articles = pd.read_csv(open('dataset/ig_article/store_article/'+fileName+'.csv')) 

        # remove '\n', delete non-string
        all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

        # get each comment's score
        all_comments_scores = get_each_comment_score(all_comments)

        # get store rating
        # print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
        store_rating = get_store_score(likes, all_comments_scores, assigned_index)
        print('\nstore final score:',store_rating) # weight: likes
        all_store_rating.append(store_rating)
        all_store_post_count.append(len())
        
    return all_store_rating



#===================================== main function =======================================
# store_names = ['鬼金棒_中山北路','雞吉君拉麵','山嵐拉麵','你回來啦 Okaeri 拉麵','麵屋壹慶','勝王','五之神製作所',
#               '柑橘 Shinn','麵屋壹の穴 ICHI']
store_names = ['鬼金棒_中山北路','雞吉君拉麵','山嵐拉麵_公館店_台灣總店','你回來啦 Okaeri 拉麵','麵屋壹慶','勝王','五之神製作所',
              '柑橘 Shinn','麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店']

#====== update senti_dict & jieba_dict ======
slef_extend_senti_dict(word_senti_score) 
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict

all_store_rating = cal_all_store_rating(store_names)

print(all_store_rating)



comment: 敲好ㄘ🌶🌶🌶🌶🍜
敲好 0.4
ㄘ 0.15
🌶 0.1
🌶 0.1
🌶 0.1
🌶 0.1
🍜 0.15
sent_score: 1.0999999999999999

comment: #🍜#ラーメン#ラーメン大好き#食べ物#カラシビ，如今本寶寶也是嚐過黑碗的人了！，（有玉米筍增量）
🍜 0.15
大好 0.43823609999999996
增量 0.2
sent_score: 0.7882361

comment: 豪豪ㄘ 嘴唇超麻～～～
豪豪 0.25
ㄘ 0.15
sent_score: 0.4

comment: 2019.12.6，第二桌！，起步～走！，，#302r4b1c ，#TY ，#秉鈞的築夢人生
起步 0.0886419
築夢 0.3
sent_score: 0.3886419

comment: #nightmarket #taiwan #taipei #noodles #stinkytofu 🤤#squid #seafood #dumplings #ningxia #nightlife
🤤 0.15
sent_score: 0.15

comment: 這次選擇辣小量、麻普通，下次可以吃辣普通、麻普通～，，#台北美食 #排隊美食 #拉麵 #鬼金棒#麻辣#中山站美食
選擇 0.0210186
普通 -0.08
下次 0.1
可以 0.1551797
普通 -0.08
普通 -0.08
美食 0.16
美食 0.16
拉麵 0.15
鬼金棒 0.25
美食 0.16
sent_score: 0.9161983

comment: 吃那個肥肉好爽
肥肉 -0.2
好爽 0.5
sent_score: 0.3

comment: Kikanbo 鬼金棒(きかんぼう)の味噌ラーメンが激辛でうまい！鬼金棒辣麻味增拉麵，麵條Q湯頭棒，直接挑戰辣麻增量過關，好吃過癮！下回可以再來挑戰辣麻鬼增量，讚！^^
鬼金棒 0.25
鬼金棒 0.25
拉麵 0.15
棒 0.13
直接 0.326906
挑戰 0.0
增量 0.2
過關 0.0051143000000000004
好吃 0.37684395
過癮 0.35
可以 0.1551797
挑戰 0.0
增量 0.2
讚 0.25
sent_score: 2.64404395000000

成熟 0.4858607
許多 0.48742399999999997
改變 0.06949739999999999
上來 0.05543455
想 0.005
歷練 0.1425984
差距 -0.35
原來 0.0
還有 0.2078135
許多 0.48742399999999997
技巧 0.28997490000000004
智慧 0.395594
看來 0.0
加上 0.16797105
了解 0.36932209999999993
其實 0.38113745
興奮 0.6946368
不光 0.0
差異 -0.57738575
包括 0.05
最後 -0.0175984
一定 0.2731792
推薦 0.019754
美食 0.16
拉麵 0.15
大概 0.045549900000000004
算是 0.15
好吃 0.37684395
拉麵 0.15
拉麵 0.15
sent_score: 5.074299600000002

comment: 人生自己選擇，選擇感受美好還是糾結困擾，，生命要浪費在美好的事物和真正重要的人物上，，那些不重要的，不屬於我的，不能放在心上，，嘿！要過好自己的人生呀👊，Keep eating, keep loving, keep moving, keep enjoying your life，#ni小猴的人生歪理 ，#ni小猴就愛吃，#名古屋コメダ珈琲早餐台北店，#全家芒果霜淇淋，#雞吉君拉麵
自己 0.0
選擇 0.0210186
選擇 0.0210186
美好 0.8176861999999999
還是 0.1
困擾 0.0
浪費 -0.27759375
美好 0.8176861999999999
真正 0.8
重要 0.33528095
重要 0.33528095
屬於 0.262271
不能 -0.060934800000000004
自己 0.0
愛吃 0.2
拉麵 0.15
sent_score: 3.52171395

comment: 開車去內湖做完健檢順道前往內湖知名雞湯拉麵店雞吉君，排了一個多小時，然後想吃的雞白湯完售，只好點鹽味雞清湯拉麵，麵體普通，但續麵一球只要10元，餛飩普通，豬叉燒不肥不柴不薄，雞肉有炙燒不柴不薄，湯頭略油略鹹，但是會想喝完的那種，環境跟服務佳，整體而言沒有花時間排隊的必要，

豐盛 0.2
白湯 0.15
sent_score: 0.35

comment: 藏身在巷內的北海道札幌豚骨拉麵，豬背脂湯底，油量蠻多（可調整），適合喜歡濃郁口味，今天吃白湯、海湯，忘了要加蛋 😆，#山嵐拉麵 #公館美食
拉麵 0.15
調整 0.37553825
適合 0.775
喜歡 0.875
白湯 0.15
山嵐拉麵 0.25
美食 0.16
sent_score: 2.73553825

comment: #勉強頑張る #ラーメン#豚骨ラーメン#山嵐ラーメン 🍜🍥🍺
勉強 0.0
山嵐 0.25
🍜 0.15
🍥 0.05
🍺 0.05
sent_score: 0.5

comment: 🍜
🍜 0.15
sent_score: 0.15

comment: The most comforting moment is the hot soup warms your heart!，.，.，.，#ramen #foodie #taiwanfood #ramennoodles #travel #foodporn #foodphotography
ramen 0.15
sent_score: 0.15

comment: 美好的晚餐❤️，湯頭很濃郁，肉片給的也很大方，偏油一點（我吃的已經只有60%的油了），聽說日本北部的拉麵油都特多是為了保熱？
美好 0.8176861999999999
很大 0.0955499
已經 0.029755599999999997
只有 0.0
聽說 0.0870222
拉麵 0.15
sent_score: 1.1800138999999998

comment: 2019.12.05，很冷很濕。，#需要很多羊毛衫，-，，二訪的#山嵐拉麵 ，覺得比上回又更好吃了！，湯頭選少油，麵條偏硬，連叉燒都肥瘦適中，就像來到日本一樣。，#整體表現高分，#白湯 #豬背脂 ，#拉麵 #山嵐 #公館站 #公館 #ramen #taipei #taiwan ，#萊斯莉的拉麵排行榜 ，勝王=山嵐>一蘭>鬼金棒=凪Nagi>鳥人>博多>一風堂>樂麵屋>鷄二>特濃屋>屯京=誠屋=麵屋一燈=山頭火>鷹流東京醬油
很多 0.28742399999999996
二訪 0.2
山嵐拉麵 0.25
覺得 0.5868026
好吃 0.37684395
叉燒 0.0


拉麵 0.15
看不見 -0.183355233
淹沒 0.0
夾起 0.0
泡沫 0.0
不易 -0.40401929999999997
可以 0.1551797
保持 0.175652
拉麵 0.15
溫度 0.0
實在 0.45718945
有名 0.36324805
想必 0.1311727
叉燒 0.0
叉燒 0.0
開始 0.1162332
軟嫩 0.1
叉燒 0.0
不僅 -0.3304674
還有 0.2078135
嫩嫩 0.0
推薦 0.019754
叉燒 0.0
白湯 0.15
更是 0.1
吃驚 -0.27323595
濃厚 0.3
富有 0.6328135
香氣 0.2
濃厚 0.3
甚至 -0.05
覺得 0.5868026
厲害 0.3
想要 0.27222515
一樣 0.0540008
務必 0.08111310000000001
拉麵 0.15
好吃 0.37684395
美味 0.6096725000000001
白湯 0.15
叉燒 0.0
濃厚 0.3
孤獨 -0.1488884
美食 0.16
ramen 0.15
sent_score: 7.364756216999999

comment: #191217
sent_score: 0

comment: 在台灣終於吃到足以抗衡Afuri清爽型醬油拉麵，筍乾有夠長,同時有炙燒叉燒和炙燒雞胸肉,CP破表🤩，#台北美食 #中山美食 #拉麵 #麵屋壹慶 #清湯醬油拉麵 #炙燒叉燒 #炙燒雞胸肉 #ramen
終於吃到 0.3
足以 0.24420245
抗衡 0.0
拉麵 0.15
有夠 0.0
同時 0.23198600000000003
叉燒 0.0
美食 0.16
美食 0.16
拉麵 0.15
拉麵 0.15
叉燒 0.0
ramen 0.15
sent_score: 1.6961884499999997

comment: 近期有人問有什麼好吃的，只想大力推薦，麵屋壹慶的雞白湯拉麵啊，，麵、湯、顏值都完美的一碗🤣，，#台北美食 #拉麵控 #拉麵 #台北拉麵 #拉麵推薦 #拉麵好好吃 #吃的是靈魂#晴光市場 #美食 #麵屋壹慶 #日本拉麵 #雞白湯拉麵 #海邊
好吃 0.37684395
只想 0.1261116
大力 0.36925559999999996
推薦 0.

sent_score: 0.136000533

comment: 今日午餐 #勝王 #拉麵
拉麵 0.15
sent_score: 0.15

comment: 其實我不喜歡吃拉麵的，但回頭想想每個月都至少會吃一次，⋯⋯⋯？，#我跟不上自己的日常 ，#薩索雞白湯拉麵
其實 0.38113745
喜歡 0.875
拉麵 0.15
至少 0.0
自己 0.0
白湯 0.15
拉麵 0.15
sent_score: 1.7061374499999997

comment: 基本款已經夠好吃了再加上黑松露，完全不想說話直到見底。，.，#food #foodie #foorporn #ramen #japanesefood #instadaily #instagood #instapic #igfood #ラーメン #delicious #叉燒 #igers #foodstagram #手機食先 #instadaily #noodles #likeforlikes #대만 #일상 #타이페이 #타이페이여행 #ぼんじり #sassochicken #勝王 #チャーシュー #黑松露雞白湯手揉麵 #onthetable #鶏白湯 #라멘 #blacktruffle
基本 0.05
已經 0.029755599999999997
好吃 0.37684395
加上 0.16797105
完全 0.45943999999999996
不想 -0.2043558
直到 0.3714897
ramen 0.15
instagood 0.2
叉燒 0.0
白湯 0.15
白湯 0.15
sent_score: 1.9011444999999998

comment: #獸王瑪莎拉香料烤蕃茄肉醬拌麵 NT.300，牛豬雞搭配特殊香料混合的肉醬，叉燒有點不ok
搭配 0.0289814
特殊 0.2399749
叉燒 0.0
sent_score: 0.2689563

comment: 吃不膩的勝王，每次都不一樣
一樣 0.0540008
sent_score: 0.0540008

comment: #肉骨茶濃湯手揉麵，#梅花叉燒，/，如其名就是肉骨茶湯底的拉麵，，加上手揉麵。，濃湯不過吃起來完全沒有負擔，，久違地加了一球麵。，以前吃濃湯麵都不太加麵的，，因為湯一下就被麵給吸光惹。，梅花叉

🍜 0.15
sent_score: 0.6

comment: 昨天吃的 哈，#豚骨叉燒拉麵🍜 ，#吃久有點鹹 😅，#東京有店喔🌱🌱🌱，#下次點沾麵看看會不會比較不鹹😆
叉燒 0.0
拉麵 0.15
🍜 0.15
下次 0.1
沾麵 0.15
不會 -0.27884699999999996
sent_score: 0.2711530000000001

comment: @bobchen0107 →招牌沾麵，我→蝦醬豚骨番茄沾麵，先說結論：嘴巴破洞真的不適合吃拉麵(；´Д｀)，大概是★★★☆☆，①番茄的比較稀釋還不錯，原味的真的...太鹹ㄌ...，②拉麵附法國麵包很有趣，然後沾了番茄蝦醬之後啃起來根本蝦味鮮，③附了青醬我有點不懂到底是幹嘛用的，應該是單純配麵吃不夠也頗鹹，青醬的蒜味很重但是我不愛蒜味(´ . .̫ . `)，④叉燒很讚，不過單純吃比較好吃，泡在沾醬裡面的叉燒丁就有點泡太久過鹹，而且有些比較邊邊角的肉老了，⑤糖心蛋相較之下沒有味道，但是配一點沾醬吃超好吃，⑥隔壁的燙青菜很解膩，解膩解膩，⑦真的好膩，，總體而言...應該不太會再來吃吧... 「我覺得這間有過敏的人應該不能來，光是空氣可能就讓人受不了了」，「嗯？？這間是蝦蝦沾醬專賣店，哪個過敏的人會找死跑來這間店？？？」
沾麵 0.15
沾麵 0.15
破洞 -0.5
適合 0.775
拉麵 0.15
大概 0.045549900000000004
不錯 0.2
拉麵 0.15
有趣 0.5578135000000001
根本 0.02277495
應該 0.016946400000000004
單純 0.075
不夠 -0.6508126000000001
叉燒 0.0
很讚 0.3
不過 0.0
單純 0.075
好吃 0.37684395
叉燒 0.0
太久 -0.07348110000000001
有些 0.25781350000000003
沒有 -0.0921865
味道 0.0
超好吃 0.6
應該 0.016946400000000004
覺得 0.5868026
過敏 -0.0745652
應該 0.016946400000000004
可能 0.0
受不了 -0.1922876
過敏 -0.0745652
sent_score: 2.8655394000000007

comment


comment: 秋刀魚拉麵🍜。
拉麵 0.15
🍜 0.15
sent_score: 0.3

comment: 目前在國內吃到最喜歡的沾麵😚😚😚
喜歡 0.875
沾麵 0.15
sent_score: 1.025

comment: 好久沒來了!!!!!!
sent_score: 0

comment: 完食，#ramen🍜 #menyaichi #tonkotsugyokai #karaage #完食 #穴穴你 #emptybowl
ramen 0.15
🍜 0.15
sent_score: 0.3

comment: 𝟚𝟘𝟙𝟡.𝟙𝟚.𝟘𝟙 #重溫前男友的口袋名單系列🍜，🍜麵屋壹の穴ICHI：煮干拉麵+雞叉燒，💰NT.250+50元，📍台北市大安區延吉街137巷22號，🚇鄰近捷運國父紀念館站1號出口，🥢梅漬聖女必推，⭐️3／5，-，#不負責任小心得🗣，壹の穴應該是前男友帶我去過所有拉麵店我最喜歡的了，分手後我自己也常常去吃，但平常都吃雞白湯；今天六點去排隊的時候老闆跟我說雞白湯賣完了⋯⋯最愛梅漬聖女小蕃茄今天也沒有⋯⋯整個很受打擊（心痛）。，但今天改口味嘗試吃了煮干拉麵，是沒有想像中的鹹，但還是更喜歡雞白湯啦。，-，#前男友小故事🐏，壹の穴是前男友帶我吃的第二間拉麵，第一間是鬼金棒。吃這兩間的時候我們還沒在一起，處於一種微妙的曖昧狀態。，要搭捷運的時候他問我今天想吃怎樣的拉麵，翻出他的口袋名單跟我一個一個介紹，最後選擇了壹の穴。，在捷運上他一口篤定的說，我一定會喜歡吃這間以及它的梅漬聖女，我半信半疑的說「真的嗎」，結果後來真的被他說中，我真的很喜歡這間的雞白湯還有梅漬聖女。，那時候他邊吃邊問我怎麼樣，我當時應該真的是眼冒愛心的說「好喜歡」。，印象中他笑得滿開心的，說著會幫我找到我最喜歡的拉麵店。我也就真的相信他會帶我去吃很多拉麵。，，有時候自己去吃壹の穴的時候還是不免會想起他，也會看著我們當時一起坐的位置發呆。說真的還是很想念他帶我來吃壹の穴的那一天。
重溫 0.0
🍜 0.15
🍜 0.15
拉麵 0.15
叉燒 0.0
出口 0.0
不負責任 -0.171182975
小心 0.0
應該 0.016946400000000004
所有 0.25781350000000003
拉麵 0.15
喜歡 0.875
分手 -0.133801
自己 0.0
常常

沾麵 0.15
終於 0.162271
拉麵 0.15
覺得 0.5868026
盡量 0.13696245
避開 -0.08396189999999999
大致 0.32272990000000007
增多 0.2704776
下去 0.0
選擇 0.0210186
只有 0.0
加上 0.16797105
使用 0.24782015
選擇 0.0210186
辛苦 0.0
選擇 0.0210186
最多 0.38002240000000004
不過 0.0
沒有 -0.0921865
特殊 0.2399749
苦味 -0.0875148
不是 -0.7609348000000001
愛 0.2
喜歡 0.875
不過 0.0
使用 0.24782015
不是 -0.7609348000000001
特別 0.31378590000000006
大概 0.045549900000000004
推測 0.030246
選擇 0.0210186
其實 0.38113745
味道 0.0
相當 0.46013175
叉燒 0.0
其實 0.38113745
一樣 0.0540008
味道 0.0
可以 0.1551797
味道 0.0
相當 0.46013175
不錯 0.2
不會 -0.27884699999999996
有些 0.25781350000000003
可以 0.1551797
直接 0.326906
超好吃 0.6
可以 0.1551797
發現 0.11151989999999998
完全 0.45943999999999996
沾麵 0.15
唯一 0.0540008
不同 0.0
流程 0.0
嘗試 0.05
知道 0.529169
直接 0.326906
其實 0.38113745
可以 0.1551797
特殊 0.2399749
不得不 0.0
控制 0.0
避免 0.0
完全 0.45943999999999996
蓋住 0.0
味道 0.0
算是 0.15
特殊 0.2399749
重點 0.1391674
根本 0.02277495
自助 0.4579737
加入 0.0625365
沾麵 0.15
當作 0.08317395
淨空 0.0
喜歡 0.875
覺得 0.5868026
好吃 0.37684395
相似 0.184672

In [963]:
#====== get rank ======
store_rank = []
temp_rating = all_store_rating[:]
temp_rating.sort(reverse = True)
for name,i in zip(store_names,all_store_rating):
    rank = temp_rating.index(i)+1
    store_rank.append(rank)
    print(name,i,rank)
    

    
#====== MSE ======
def MSE(y, y_pred):
 
    mse = np.mean((y - y_pred)**2)
   
    return mse

print('\nstore_rank:',store_rank)
print('MSE:',MSE(np.array(range(1,10)),np.array(store_rank)))

鬼金棒_中山北路 0.6932746468831456 7
雞吉君拉麵 0.9116819486125125 1
山嵐拉麵_公館店_台灣總店 0.677048314503546 8
你回來啦 Okaeri 拉麵 0.877502460831607 2
麵屋壹慶 0.8118209487970253 4
勝王 0.7800336697581352 6
五之神製作所 0.7898212323118503 5
柑橘 Shinn 0.8707924849063174 3
麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店 0.6613204122823985 9

store_rank: [7, 1, 8, 2, 4, 6, 5, 3, 9]
MSE: 10.666666666666666


In [626]:
# google ramen rating
print(MSE(np.array(range(1,10)),np.array([9,3,5,4,1,6,7,2,8])))

13.555555555555555


In [960]:
# store_names = ['辣麻味噌拉麵 鬼金棒','雞吉君拉麵','山嵐拉麵','你回來啦 Okaeri 拉麵']
# store_page_xpath = ['href="/explore/locations/541098880/"','href="/explore/tags/雞吉君拉麵/"','href="/explore/locations/610378299165578/"','href="/explore/locations/1009572868/okaeri/"']
# first_page_xpath = ['href="/p/B-Rf46-jXl_/"','href="/p/B-CEZjVl8GC/"','href="/p/B-Wy4mhnnvk/"', 'href="/p/B-KNu4oFwcv/"']
# get_page_count = [500,130,500,500]


# store_names = ['一蘭拉麵','麵屋壹の穴 ICHI','勝王']
# store_page_xpath = ['href="/explore/locations/255951018612300/"','href="/explore/locations/1719618628292660/ichi/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-ZW6yUASZL/"','href="/p/B-ZXrQEnbZ-/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [1200,1000,1000]


# store_names = ['一蘭拉麵']
# store_page_xpath = ['href="/explore/locations/255951018612300/"']
# first_page_xpath = ['href="/p/B-ZW6yUASZL/"']
# get_page_count = [4000]


# store_names = ['柑橘 Shinn','五之神製作所 台灣','麵屋壹慶','勝王']
# store_page_xpath = ['href="/explore/locations/765135860311072/shinn/"','href="/explore/locations/1919809258303714/"','href="/explore/locations/308926743186267/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-ZpZrvnZru/"','href="/p/B-Y78zMBrmj/"','href="/p/B-ZM0FnJglR/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [600,700,700,700]

# store_names = ['五之神製作所 台灣','麵屋壹慶','勝王']
# store_page_xpath = ['href="/explore/locations/1919809258303714/"','href="/explore/locations/308926743186267/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-Y78zMBrmj/"','href="/p/B-ZM0FnJglR/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [700,700,700]

store_selenium_param = get_selenium_param('selenium_url')
store_names = []
store_page_xpath = []
first_page_xpath = []

for line in store_selenium_param:
    store_names.append(line[0])
    store_page_xpath.append(line[1])
    first_page_xpath.append(line[2])

    
print(first_page_xpath)
# store_names = [store_selenium_param[0][0]]
# store_page_xpath = [store_selenium_param[0][1]]
# first_page_xpath = [store_selenium_param[0][2]]
# get_page_count = [4000]
# print(first_page_xpath)

['href="/p/B_AJAlQnfqq/"', 'href="/p/B-9s0xmB9Ds/"']


In [961]:
#===== scrape url from assigned date ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(4)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(4)

    # get url of current page
    url_list, post_date, output_post_date = [], [], []
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]) # get first page date

    # get 500 page url
    for i in range(4000):
        
        stopDate = post_date[-1][0]
        
        if stopDate[:stopDate.find('年')] != '2019':
#             print(stopDate)
            
            if check_exists_by_xpath('//a[@tabindex="0"][2]'):
                driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
                time.sleep(3)
            else:
                break
            
            post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
            continue
            
            
#         time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(1)
        temp_date = [driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]
        post_date.append(temp_date)
        output_post_date.append(temp_date)
        
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
            time.sleep(4)
        else:
            break

        # stop date = 2019.8.1
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '7':
            break
        
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list[:-1], 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], output_post_date[:-1], 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print((stopTime-startTime)/60)

麵屋壹の穴 ICHIfinished!
麵屋壹の穴ichi-沾麵專門店finished!
27.987660384178163


In [400]:
#===== scrape url from first post ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(4)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(3)

    # get url of current page
    url_list, post_date = [], []
    url_list.append([driver.current_url]) # get first page url
    time.sleep(1)
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
    time.sleep(3)

    # get 500 page url
#     flag = 0
#     for i in range(get_page_count[store_index]):
    for i in range(4000):
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click()
        else:
            break

        time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(5)
        post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
        stopDate = post_date[-1][0]
#         print(post_date[-1][0],type(post_date[-1][0]))
        
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '7':
            break
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list, 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], post_date, 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print(stopTime-startTime)

這一鍋皇室秘藏鍋物finished!
1462.405298948288


In [259]:
# teststr = '2019年8月10日'
teststr = '3月23日'
if teststr[:teststr.find('年')] == '2019' and teststr[teststr.find('年')+1:teststr.find('月')] == '8':
    print('yes')

# extend dictionary

In [413]:
# extend dictionary
word_to_weight = {
    "土地公": 1,
    "土地婆": 1,
    "公有": 2,
    "緯來體育台": 1,
    "鬼金棒": 3,
    "海底撈": 5
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(2, {'公有': 2.0}), (3, {'土地公': 1.0, '土地婆': 1.0, '鬼金棒': 3.0, '海底撈': 5.0}), (5, {'緯來體育台': 1.0})]
